In [ ]:
!pip install -q streamlit google-generativeai gTTS PyPDF2 pyngrok
!pip uninstall -y google-generativeai -q
!pip install -qU google-generativeai==0.8.5


In [ ]:
%%writefile tools.py
import google.generativeai as genai
from gtts import gTTS
import tempfile, io
from PyPDF2 import PdfReader

def configure_gemini(api_key: str):
    genai.configure(api_key=api_key)

def extract_pdf_text(uploaded_file) -> str:
    """Extracts text locally so we only send TEXT to the model (no file API)."""
    try:
        data = uploaded_file.getvalue()
        reader = PdfReader(io.BytesIO(data))
        text = "\n".join([(p.extract_text() or "") for p in reader.pages]).strip()
        # compact for context
        if len(text) > 12000:
            text = text[:6000] + "\n...\n" + text[-5800:]
        return text
    except Exception as e:
        print("Text Extraction Error:", e)
        return ""

def google_text_to_speech(text: str):
    try:
        tts = gTTS(text=text, lang='en', slow=False)
        tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
        tts.save(tmp.name)
        return tmp.name
    except Exception as e:
        print("TTS Error:", e)
        return None


Overwriting tools.py


In [ ]:
%%writefile agent_logic.py
import google.generativeai as genai
import json

# ----- model state -----
_MODEL_ID = None

def set_model_id(mid: str):
    """Called by the UI after you pick a model from the dropdown."""
    global _MODEL_ID
    _MODEL_ID = mid

def list_supported_models():
    """Return (id, label) for models that support generateContent."""
    out = []
    for m in genai.list_models():
        methods = getattr(m, "supported_generation_methods", []) or []
        if "generateContent" in methods:
            # Prefer the full resource name if present
            out.append(m.name)
    # Keep unique and sorted, most human-friendly last part
    out = sorted(set(out), key=lambda s: s.split("/")[-1])
    return out

def get_model():
    if not _MODEL_ID:
        raise RuntimeError("Model not selected. Call set_model_id() first.")
    return genai.GenerativeModel(_MODEL_ID)

# ---------- Recruiter (text only) ----------
def recruiter_agent(role, resume_text="", candidate_intro=""):
    model = get_model()
    ctx = resume_text or candidate_intro or "No context provided."
    sys = (f"You are a recruiter for {role}. "
           "Greet the candidate by name if available from the résumé and ask the FIRST technical question "
           "tailored to their skills. Keep it concise and specific.")
    prompt = f"{sys}\n\nRésumé/Context:\n{ctx}"
    return model.generate_content(prompt).text

def recruiter_followup(role, candidate_answer, step):
    model = get_model()
    prompt = (
        f"Candidate answered: {candidate_answer}\n"
        f"Current Step: {step}/4.\n"
        "If step < 4: acknowledge, then ask the next deeper technical question based on their résumé.\n"
        "If step == 4: thank them and say 'Interview Complete'. Keep it concise."
    )
    return model.generate_content(prompt).text

# ---------- Trainer (communication JSON) ----------
def trainer_agent(role, candidate_answer):
    model = get_model()
    prompt = (
      f"You are a personal interview trainer for the role {role}. "
      "Evaluate the candidate's answer for clarity, confidence, structure (STAR) and tone. "
      "Return ONLY valid minified JSON with this schema:\n"
      '{"clarity":0,"confidence":0,"structure":0,"tone":0,'
      '"praise":[],"improvements":[],"drill":""}\n'
      f"Answer: {candidate_answer}"
    )
    out = model.generate_content(prompt).text
    try:
        return json.loads(out)
    except:
        return {"raw_output": out}

# ---------- Evaluator (final report JSON) ----------
def evaluator_agent(chat_history):
    model = get_model()
    conversation = "\n".join([f"{m['role']}: {m['content']}" for m in chat_history][-12000:])
    prompt = (
      "Evaluate the candidate based on the conversation.\n"
      "Return ONLY valid minified JSON exactly matching:\n"
      '{"candidate_name":"",'
      '"technical_score":0,"communication_score":0,"confidence_score":0,'
      '"overall":0,"strengths":[],"improvements":[],"verdict":""}\n'
      f"Conversation:\n{conversation}"
    )
    out = model.generate_content(prompt).text
    try:
        return json.loads(out)
    except:
        return {"raw_output": out}

# ---------- Improvement Plan (skills gap + 4-week plan JSON) ----------
def improvement_plan_agent(role, chat_history, resume_text_snippet=""):
    model = get_model()
    convo = "\n".join([f"{m['role']}: {m['content']}" for m in chat_history][-6000:])
    prompt = (
      f"You are a senior mentor creating a skills-gap report and a 4-week plan for a {role} candidate.\n"
      "Use the resume/context and conversation to infer strengths and gaps.\n"
      "Return ONLY valid minified JSON with this schema:\n"
      '{"skills_gaps":[],'
      '"recommended_courses":[],'
      '"practice_projects":[],'
      '"daily_drills":[],'
      '"week_by_week_plan":[{"week":1,"focus":"","tasks":[]},{"week":2,"focus":"","tasks":[]},{"week":3,"focus":"","tasks":[]},{"week":4,"focus":"","tasks":[]}]}'
      f"\nResume/context:\n{resume_text_snippet}\n\nConversation:\n{convo}"
    )
    out = model.generate_content(prompt).text
    try:
        return json.loads(out)
    except:
        return {"raw_output": out}


Overwriting agent_logic.py


In [ ]:
%%writefile app.py
import streamlit as st
from tools import configure_gemini, extract_pdf_text, google_text_to_speech
from agent_logic import (
    recruiter_agent, recruiter_followup,
    trainer_agent, evaluator_agent, improvement_plan_agent,
    set_model_id, list_supported_models
)

st.set_page_config(page_title="TalentSphere – Intelligent Interview & Growth System", page_icon="💼", layout="wide")
st.title("💼 TalentSphere – Intelligent Interview & Growth System")
st.caption("Empowering confident hiring and personal growth | © 2025 Adnan Shami")
st.divider()

# ---------- Sidebar ----------
with st.sidebar:
    st.header("⚙️ Configuration")
    api_key = st.text_input("Google AI Studio API Key", type="password")

    # Step 1: configure & fetch models
    if api_key:
        configure_gemini(api_key)
        if st.button("🔄 Load available models"):
            st.session_state['models'] = list_supported_models()
    models = st.session_state.get('models', [])

    if models:
        chosen = st.selectbox("Choose model (from your key):", models, index=0, format_func=lambda s: s.split("/")[-1])
        set_model_id(chosen)
        st.success(f"Model selected: {chosen}")
    else:
        st.info("Enter API key and click 'Load available models' to populate the list.")

    role = st.text_input("Target Role", value="Software Engineer")
    st.markdown("**Candidate Input**")
    pdf = st.file_uploader("Upload Résumé (PDF) — Recommended", type="pdf")
    candidate_intro = st.text_area("OR type a short intro", placeholder="e.g. 2 years in Python, data pipelines, MySQL...")

    start = st.button("🚀 Start Interview", type="primary")

# ---------- Session State ----------
ss = st.session_state
ss.setdefault("chat", [])
ss.setdefault("step", 0)
ss.setdefault("active", False)
ss.setdefault("resume_text", "")
ss['role'] = role

# ---------- Start Flow ----------
if start and api_key and ('models' in ss) and ss['models']:
    resume_text = ""
    if pdf is not None:
        resume_text = extract_pdf_text(pdf)
        if resume_text:
            st.sidebar.success("Résumé parsed ✓")
    ss.resume_text = resume_text

    first_msg = recruiter_agent(role=role, resume_text=resume_text, candidate_intro=candidate_intro)
    ss.chat = [{"role": "assistant", "content": first_msg}]
    ss.step = 1
    ss.active = True
    st.rerun()

# ---------- Chat UI ----------
for msg in ss.chat:
    with st.chat_message(msg["role"]):
        st.write(msg["content"])
        if msg["role"] == "assistant":
            audio = google_text_to_speech(msg["content"])
            if audio: st.audio(audio)

# ---------- Turn-taking ----------
if user := st.chat_input("Your answer..."):
    if ss.active:
        ss.chat.append({"role": "user", "content": user})

        fb = trainer_agent(ss['role'], user)
        ss.chat.append({"role": "assistant", "content": f"🎯 Trainer Feedback: {fb}"})

        if ss.step >= 4:
            with st.spinner("Generating Final Evaluation & Growth Plan..."):
                report = evaluator_agent(ss.chat)
                plan = improvement_plan_agent(ss['role'], ss.chat, ss.resume_text[:1500])
            st.success("✅ Interview Complete!")
            st.markdown("### 📋 Final Evaluation")
            st.json(report)
            st.markdown("### 🧭 Skills-Gap & 4-Week Growth Plan")
            st.json(plan)
            ss.active = False
        else:
            response = recruiter_followup(ss['role'], user, ss.step)
            ss.step += 1
            ss.chat.append({"role": "assistant", "content": response})
        st.rerun()


Overwriting app.py


In [ ]:
from pyngrok import ngrok, conf
import subprocess, sys, time

NGROK_AUTH_TOKEN = "36BcLEmYraAvQMl5bpemdzR1nU5_7ontawGJsf4UPJQQ7Q8qf"
conf.get_default().region = "in"   # change if needed

ngrok.kill()
!pkill -f "streamlit" || true

proc = subprocess.Popen([sys.executable, "-m", "streamlit", "run", "app.py",
                         "--server.port", "8501", "--server.headless", "true"])
time.sleep(5)

ngrok.set_auth_token(NGROK_AUTH_TOKEN)
public_url = ngrok.connect(8501, "http").public_url
print("🌍 Live App URL:", public_url)


^C
🌍 Live App URL: https://calligraphic-eufemia-unliberalised.ngrok-free.dev


In [ ]:
!pip freeze | grep -E "streamlit|google-generativeai|gTTS|PyPDF2|pyngrok" > requirements.txt


TalentSphere/
│
├── app.py
├── tools.py
├── agent_logic.py
├── requirements.txt
├── README.md
└── assets/
    └── screenshot.png   (optional – e.g., your Streamlit UI image)
